# Stacking Example
#### <font color="blue"> by: Jeremy Meinke

This notebook will outline the various maps and stacking code that I've built
(and now cleaned up for you).  

The goal of everything here is to:
1.  Show a good summary of my work conducted.
2.  Provide others (future grad students or general astrophysics audience) with code that can be the foundation for even cooler analysis!

---


To start with, we'll load up the SPT-SZ Cluster Catalog.  For convenience, you can use the shell scripts in ./data/ to download any files I'll show here.  

In ./data/ terminal, run "bash pull_spt_cluster_catalog_2019_wget.sh"

Done? Okay:

In [1]:
from __future__ import division
import numpy as np
import pandas as pd
from astropy.table import Table
import healpy as hp
import matplotlib.pyplot as plt
import time
import gc  
###SPT maps are big... gc.collect() is a safety way to try to keep the notebook from using too much memory

from stacking_code import stacking    ###My stacking code

print("Last time run: %s"%time.asctime())


Last time run: Thu Jan 26 19:34:27 2023


Now reading the cluster catalog and into a pandas table:

In [2]:
catalog = Table.read("./data/2500d_cluster_sample_Bocquet19.fits", format="fits").to_pandas()
print("Table Columns: \n", list(catalog.columns))
print("\nCatalog size: ", len(catalog))
# print(catalog)    ###To view what each column looks like

Table Columns: 
 ['SPT_ID', 'RA', 'DEC', 'FIELD', 'XI', 'THETA_CORE', 'YSZ', 'DYSZ', 'REDSHIFT', 'REDSHIFT_UNC', 'REDSHIFT_LIMIT', 'XRAY', 'SPECZ', 'WEAK_LENSING', 'M500', 'M500_uerr', 'M500_lerr', 'M200', 'M200_uerr', 'M200_lerr', 'M500_marge', 'M500_marge_uerr', 'M500_marge_lerr', 'M200_marge', 'M200_marge_uerr', 'M200_marge_lerr']

Catalog size:  677


Now selecting the RAs and DECs of our catalog

In [3]:
catalog_ra = catalog["RA"]
catalog_dec = catalog["DEC"]

### Next Step: Load Map (SPT 95 GHz)
***